# Analysis of flight delay in air transportation using machine learning
Steven Corroy

The goal of this code is to demonstrate how flight delay can be analyzed and predicted using machine learning.  

# I) Libraries, data and train/test split

## A) Libraries

In [154]:
%matplotlib inline
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.feature_selection import chi2, f_classif, SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, roc_auc_score, roc_curve
from sklearn import preprocessing
from sklearn.feature_selection import RFE, RFECV
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import PredefinedSplit

In [2]:
root_directory = '/Users/estcorr/data/research/'

## B) Data

We load the 'On-Time Performance' table from transtats.bts.gov for the 12 months of 2015. There is one file per month as 1.csv for January, 2.csv for Februray, ...

This data essentially contains all the flights for the major US carriers during 2015 and their corresponding delay performance.

In [3]:
delay_list = list()
for i in range(1,3):
    delay_list.append(pd.read_csv(root_directory+str(i)+'.csv', dtype={'FL_DATE': str,
                                                                       'CRS_DEP_TIME': str, 
                                                                       'DEP_TIME': str, 
                                                                       'CRS_ARR_TIME': str, 
                                                                       'ARR_TIME': str}))
delay_all = pd.concat(delay_list)
delay_all.sort_values(by=['MONTH', 'DAY_OF_MONTH'], inplace=True)
delay_all.index = np.arange(len(delay_all))

/Library/Python/2.7/site-packages/IPython/core/interactiveshell.py:2871: DtypeWarning: Columns (77,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Library/Python/2.7/site-packages/IPython/core/interactiveshell.py:2871: DtypeWarning: Columns (77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


We load the 'T-100 Domestic Segment (U.S. Carriers)' table from transtats.bts.gov for 2015. This table contains monthly data about the amount of passengers transported by carriers on all US routes.

In [4]:
pass_list = list()
for i in range(1,3):
    pass_list.append(pd.read_csv(root_directory+'passengers_'+str(i)+'.csv'))
passengers = pd.concat(pass_list)
passengers.index = np.arange(len(passengers))

Finally we load the excel document provided for the assignment containing the latitude and longitude of all airports.

In [5]:
airports = pd.read_csv(root_directory+'airports2.csv',sep=';')
airports.long = airports.long.map(lambda x: x.replace(',','.')).astype('float')
airports.lat = airports.lat.map(lambda x: x.replace(',','.')).astype('float')

## C) Train/test split
This is one the delicate things working with time series or time dependant data. We need to split our data into train/validation and test to be able to tune and evaluate our machine learning models. A typical good way to do it for non-time-dependant would be a random 2/3 train and 1/3 test followed a randomized 10-folds cross-validation of the training set for hyper-parameter tunning. Here we cannot do that as it would introduce leakage in the data process, aka we learn from the future. For the sake of time we make a simple split as follows.
- The month of January will be used a training set
- The first 14 days of February will be used a cross-validation set
- The last 14 days of February will be used as test set
General statistics used as feature will only be calculated on the training set and daily dynamics features are calculated for all 3 sets.

NOTE: a training/learning on many months/years would surely provide more interesting/trustable results. The choice of taking only 2 months into account is purely to be able to run the code in a reasonable amount of time. Simply by changing the data loading part of this code, one could re-run the analysis on a larger dataset.

In [6]:
delay = delay_all[delay_all.MONTH == 1]
delay_cv = delay_all[(delay_all.MONTH == 2) & (delay_all.DAY_OF_MONTH <= 14)]
delay_test = delay_all[(delay_all.MONTH == 2) & (delay_all.DAY_OF_MONTH > 14)]

Accordingly for the passenger statistics we select only January

In [7]:
passengers = passengers[passengers.MONTH == 1]

Finally we extract the number of days present in the training data. This will be used all along the code to normalize metrics.

In [8]:
n_days = len(np.unique(delay['FL_DATE']))
n_days

31

# II) Features generation
Here we generate all the features that will be used for predicting delay.

GENERAL NOTE FOR THIS NOTEBOOK: many of these features could be computed together in one line (we could decrease the computation time by not repeating 'groupby' and 'join' operations), or in a more efficient/concise way. Here I prefered to disjoin every features to improve readability and for time limitation reason to use function more because the ease of use rather than efficiency (as long as running time was acceptable).

In [9]:
data = delay[['UNIQUE_CARRIER', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID', 'ORIGIN', 'DEST']]

## A) Airport features
These features are related to how the departing and arriving airports behave in general (i.e., for all routes and carriers).

### 1) Number of flights per day at the departing airport
This is the average utilization of the departing airport and could indicate how congested it is.

In [10]:
dep_flights = (delay.groupby('ORIGIN_AIRPORT_ID').MONTH.count()/n_days).to_frame('DEP_FLIGHTS')
data = pd.merge(data, dep_flights, how='inner', left_on='ORIGIN_AIRPORT_ID', right_index=True, sort=False)

### 2) Number of flights per day at the arriving airport

In [11]:
arr_flights = (delay.groupby('DEST_AIRPORT_ID').MONTH.count()/n_days).to_frame('ARR_FLIGHTS')
data = pd.merge(data, arr_flights, how='inner', left_on='DEST_AIRPORT_ID', right_index=True, sort=False)

### 3) Number of routes served by the departing airport
This gives how many different routes are offered from the airport and could indicate a large spread of risk in terms of delay but also a more challenging diversity of partners to deal with.

In [12]:
dep_routes = (delay.groupby('ORIGIN_AIRPORT_ID').DEST_AIRPORT_ID.agg(lambda x: len(np.unique(x)))).to_frame('DEP_ROUTES')
data = pd.merge(data, dep_routes, how='inner', left_on='ORIGIN_AIRPORT_ID', right_index=True, sort=False)

### 4) Number of routes ending at the arriving airport

In [13]:
arr_routes = (delay.groupby('DEST_AIRPORT_ID').ORIGIN_AIRPORT_ID.agg(lambda x: len(np.unique(x)))).to_frame('ARR_ROUTES')
data = pd.merge(data, arr_routes, how='inner', left_on='DEST_AIRPORT_ID', right_index=True, sort=False)

### 5) Number of passengers per day at the departing airport
Similar to 1) but taking more into account the flow of passengers rather than planes. To get this one we need to merge with the 'passsengers' table.

In [14]:
dep_passengers = (passengers.groupby('ORIGIN_AIRPORT_ID').PASSENGERS.sum()/n_days).to_frame('DEP_PASSENGERS')
data = pd.merge(data, dep_passengers, how='inner', left_on='ORIGIN_AIRPORT_ID', right_index=True, sort=False)

### 6) Number of passengers per day at the arriving airport

In [15]:
arr_passengers = (passengers.groupby('DEST_AIRPORT_ID').PASSENGERS.sum()/n_days).to_frame('ARR_PASSENGERS')
data = pd.merge(data, arr_passengers, how='inner', left_on='DEST_AIRPORT_ID', right_index=True, sort=False)

### 7) Rate of occupancy of the departing airport
With this we calculate how close to its maximal capacity the airport is 

In [16]:
passengers['OCCUPANCY'] = passengers.PASSENGERS/passengers.SEATS
dep_occupancy = (passengers.groupby('ORIGIN_AIRPORT_ID').OCCUPANCY.mean()).to_frame('DEP_OCCUPANCY')
data = pd.merge(data, dep_occupancy, how='inner', left_on='ORIGIN_AIRPORT_ID', right_index=True, sort=False)

### 8) Rate of occupancy of the arriving airport

In [17]:
arr_occupancy = (passengers.groupby('DEST_AIRPORT_ID').OCCUPANCY.mean()).to_frame('ARR_OCCUPANCY')
data = pd.merge(data, arr_occupancy, how='inner', left_on='DEST_AIRPORT_ID', right_index=True, sort=False)

## B) Route features
These features describe the behavior of the specific route considered in general (i.e., for all carriers)

### 1) Number of times the route is served per day
This could indicate if this route is usual for the two airports and is a well-known relationship.

In [18]:
route_frequency = (delay.groupby(['ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID']).MONTH.count()/n_days).to_frame('ROUTE_FREQ')
data = pd.merge(data, route_frequency, how='inner', left_on=['ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID'], right_index=True, sort=False)

### 2) Route importance for departing and arriving airports
This is to estimate how much percentage of the resources of the airport go to this route

In [19]:
data['ROUTE_IMP_DEP'] = data.ROUTE_FREQ/data.DEP_FLIGHTS
data['ROUTE_IMP_ARR'] = data.ROUTE_FREQ/data.ARR_FLIGHTS

### 3) Number of passengers on the route
This indicates how busy the route is

In [20]:
route_passengers = (passengers.groupby(['ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID']).PASSENGERS.sum()/n_days).to_frame('ROUTE_PASSENGERS')
data = pd.merge(data, route_passengers, how='inner', left_on=['ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID'], right_index=True, sort=False)

### 4) Route importance in terms of passengers
This is again to understand how dominating a route is for an airport but this time in terms of number of passengers. We try to catch the fact that taking care of one huge aircraft with many passengers could more work than many smaller aircrafts.

In [21]:
data['ROUTE_IMP_PASS_DEP'] = data.ROUTE_PASSENGERS/data.DEP_PASSENGERS
data['ROUTE_IMP_PASS_ARR'] = data.ROUTE_PASSENGERS/data.ARR_PASSENGERS

### 5) Route rate of occupancy
Here we calculate the average rate of filling of the route. This could indicate how close to maximum capacity the airport is when it comes to this route. Also it shows how much more crowded a flight can be compared to average for the airport.

In [22]:
route_occupancy= (passengers.groupby(['ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID']).OCCUPANCY.mean()).to_frame('ROUTE_OCCUPANCY')
data = pd.merge(data, route_occupancy, how='inner', left_on=['ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID'], right_index=True, sort=False)

## C) Carrier features
These features describe the behavior of the specific carrier considered in general (i.e., for all routes)

### 1) Number of flights by the carrier at the departing and arriving airports
This can show how busy a carrier is in terms of number of flights.

In [23]:
dep_carrier_flights = (delay.groupby(['ORIGIN_AIRPORT_ID','UNIQUE_CARRIER']).MONTH.count()/n_days).to_frame('DEP_CARRIER_FLIGHTS')
data = pd.merge(data, dep_carrier_flights, how='inner', left_on=['ORIGIN_AIRPORT_ID','UNIQUE_CARRIER'], right_index=True, sort=False)
arr_carrier_flights = (delay.groupby(['DEST_AIRPORT_ID','UNIQUE_CARRIER']).MONTH.count()/n_days).to_frame('ARR_CARRIER_FLIGHTS')
data = pd.merge(data, arr_carrier_flights, how='inner', left_on=['DEST_AIRPORT_ID','UNIQUE_CARRIER'], right_index=True, sort=False)

### 2) Number of routes served by the carrier at the departing and arriving airports
This can show how busy a carrier is in terms of number of different routes.

In [24]:
dep_carrier_routes = (delay.groupby(['ORIGIN_AIRPORT_ID','UNIQUE_CARRIER']).DEST_AIRPORT_ID.agg(lambda x: len(np.unique(x)))).to_frame('DEP_CARRIER_ROUTES')
data = pd.merge(data, dep_carrier_routes, how='inner', left_on=['ORIGIN_AIRPORT_ID','UNIQUE_CARRIER'], right_index=True, sort=False)
arr_carrier_routes = (delay.groupby(['DEST_AIRPORT_ID','UNIQUE_CARRIER']).ORIGIN_AIRPORT_ID.agg(lambda x: len(np.unique(x)))).to_frame('ARR_CARRIER_ROUTES')
data = pd.merge(data, arr_carrier_routes, how='inner', left_on=['DEST_AIRPORT_ID','UNIQUE_CARRIER'], right_index=True, sort=False)

### 3) Number of passengers transported by the carrier at the departing and arriving airports
This can show how busy a carrier is in terms of passengers transported.

In [25]:
dep_carrier_passengers = (passengers.groupby(['ORIGIN_AIRPORT_ID','UNIQUE_CARRIER']).PASSENGERS.sum()/n_days).to_frame('DEP_CARRIER_PASSENGERS')
data = pd.merge(data, dep_carrier_passengers, how='inner', left_on=['ORIGIN_AIRPORT_ID','UNIQUE_CARRIER'], right_index=True, sort=False)
arr_carrier_passengers = (passengers.groupby(['DEST_AIRPORT_ID','UNIQUE_CARRIER']).PASSENGERS.sum()/n_days).to_frame('ARR_CARRIER_PASSENGERS')
data = pd.merge(data, arr_carrier_passengers, how='inner', left_on=['DEST_AIRPORT_ID','UNIQUE_CARRIER'], right_index=True, sort=False)

### 4) Carrier importance
Here we estimate how much of the traffic is to account for a carrier at the arriving and receiving airport in terms of flights, routes and passengers transported.

In [26]:
data['CARRIER_IMP_DEP'] = data.DEP_CARRIER_FLIGHTS/data.DEP_FLIGHTS
data['CARRIER_IMP_ROUTE_DEP'] = data.DEP_CARRIER_ROUTES/data.DEP_ROUTES
data['CARRIER_IMP_PASS_DEP'] = data.DEP_CARRIER_PASSENGERS/data.DEP_PASSENGERS
data['CARRIER_IMP_ARR'] = data.ARR_CARRIER_FLIGHTS/data.ARR_FLIGHTS
data['CARRIER_IMP_ROUTE_ARR'] = data.ARR_CARRIER_ROUTES/data.ARR_ROUTES
data['CARRIER_IMP_PASS_ARR'] = data.ARR_CARRIER_PASSENGERS/data.ARR_PASSENGERS

### 5) Rate of occupancy for all the flights of a carrier at the departing and arriving airport
This measure should account for the activity of a carrier at an airport.

In [27]:
dep_carrier_occupancy = (passengers.groupby(['ORIGIN_AIRPORT_ID','UNIQUE_CARRIER']).OCCUPANCY.mean()).to_frame('DEP_CARRIER_OCCUPANCY')
data = pd.merge(data, dep_carrier_occupancy, how='inner', left_on=['ORIGIN_AIRPORT_ID','UNIQUE_CARRIER'], right_index=True, sort=False)
arr_carrier_occupancy = (passengers.groupby(['DEST_AIRPORT_ID','UNIQUE_CARRIER']).OCCUPANCY.mean()).to_frame('ARR_CARRIER_OCCUPANCY')
data = pd.merge(data, arr_carrier_occupancy, how='inner', left_on=['DEST_AIRPORT_ID','UNIQUE_CARRIER'], right_index=True, sort=False)

## D) Carrier and route features
These features relate to the performance of a specific carrier on a specific route

### 1) Number of time that the carrier flies the route per day
This can describe how regular a route is for a carrier, which could mean being more used to fly the route but also more prone to delay avalanche.

In [28]:
route_carrier_frequency = (delay.groupby(['ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','UNIQUE_CARRIER']).MONTH.count()/n_days).to_frame('ROUTE_CARRIER_FREQ')
data = pd.merge(data, route_carrier_frequency, how='inner', left_on=['ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','UNIQUE_CARRIER'], right_index=True, sort=False)

### 2) Number of passengers per day transported by the carrier on the route
How busy the specific route is for the carrier

In [29]:
route_carrier_passengers = (passengers.groupby(['ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','UNIQUE_CARRIER']).PASSENGERS.sum()/n_days).to_frame('ROUTE_CARRIER_PASSENGERS')
data = pd.merge(data, route_carrier_passengers, how='inner', left_on=['ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','UNIQUE_CARRIER'], right_index=True, sort=False)

### 3) Rate of occupancy of the route for the carrier
How full the specific route is for the carrier

In [30]:
route_carrier_occupancy= (passengers.groupby(['ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','UNIQUE_CARRIER']).OCCUPANCY.mean()).to_frame('ROUTE_CARRIER_OCCUPANCY')
data = pd.merge(data, route_carrier_occupancy, how='inner', left_on=['ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','UNIQUE_CARRIER'], right_index=True, sort=False)

### 4) Importance of the route for the carrier wrt. number of passengers and number 
Here we not only compare to the average with the whole airport but also for the route in general and the carrier in general 

In [31]:
data['ROUTE_CARRIER_IMP_DEP'] = data.ROUTE_CARRIER_FREQ/data.DEP_FLIGHTS
data['ROUTE_CARRIER_IMP_ARR'] = data.ROUTE_CARRIER_FREQ/data.ARR_FLIGHTS
data['ROUTE_CARRIER_IMP_PASS_DEP'] = data.ROUTE_CARRIER_PASSENGERS/data.DEP_PASSENGERS
data['ROUTE_CARRIER_IMP_PASS_ARR'] = data.ROUTE_CARRIER_PASSENGERS/data.ARR_PASSENGERS

data['ROUTE_CARRIER_ROUTE_IMP'] = data.ROUTE_CARRIER_FREQ/data.ROUTE_FREQ
data['ROUTE_CARRIER_ROUTE_IMP_PASS'] = data.ROUTE_CARRIER_PASSENGERS/data.ROUTE_PASSENGERS

data['ROUTE_CARRIER_CARRIER_IMP_DEP'] = data.ROUTE_CARRIER_FREQ/data.DEP_CARRIER_FLIGHTS
data['ROUTE_CARRIER_CARRIER_IMP_ARR'] = data.ROUTE_CARRIER_FREQ/data.ARR_CARRIER_FLIGHTS
data['ROUTE_CARRIER_CARRIER_IMP_PASS_DEP'] = data.ROUTE_CARRIER_PASSENGERS/data.DEP_CARRIER_PASSENGERS
data['ROUTE_CARRIER_CARRIER_IMP_PASS_ARR'] = data.ROUTE_CARRIER_PASSENGERS/data.ARR_CARRIER_PASSENGERS

## E) Geography
Here we capture the localization of the 2 airports on a route to catch typical weather effect depending on geography. We get long and lat from one of the 2 provided table.

In [32]:
data = pd.merge(data, airports[['iata','lat','long']], how='inner', left_on='ORIGIN', right_on='iata', sort=False)
data = pd.merge(data, airports[['iata','lat','long']], how='inner', left_on='DEST', right_on='iata', sort=False)
data.drop(['iata_x','iata_y','ORIGIN','DEST'], inplace=True, axis=1)

## F) Putting general statistics together in CV and test sets
All the above features have been calculated on the training set and will be reused as such for the CV and test sets. We simply join the CV and test set with the obtained features.

In [33]:
data_cv = delay_cv[['UNIQUE_CARRIER', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID']]
data_test = delay_test[['UNIQUE_CARRIER', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID']]

In [34]:
general_features = data.groupby(['UNIQUE_CARRIER', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID']).first()

In [35]:
data_cv = pd.merge(data_cv, general_features, how='inner', left_on=['UNIQUE_CARRIER', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID'], right_index=True, sort=False)
data_test = pd.merge(data_test, general_features, how='inner', left_on=['UNIQUE_CARRIER', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID'], right_index=True, sort=False)

In [36]:
n_train_samples = data.shape[0]
n_cv_samples = data_cv.shape[0]
n_test_samples = data_test.shape[0]

print 'training set size=',n_train_samples,', cv set size=',n_cv_samples,', test set size=',n_test_samples

data = pd.concat([data, data_cv, data_test])
print(data.shape)

training set size= 469397 , cv set size= 210405 , test set size= 216414
(896216, 49)


## F) Time (Periodic effects)
Here we capture the general time periodicity with the day of the year and the day of the month. That should help catch typical busy days and typical cold/windy months

In [37]:
data['DAY'] = delay_all.DAY_OF_WEEK
data['MONTH'] = delay_all.MONTH

## G) Daily time dynamic
This is the hard part. Here we want to capture the effect of delay building up gradually, e.g., in an airport under bad weather condition or strike, or for a carrier that has slight delay in the beginning of the day but tight margins and therefore higher and higher delay as time passes. We will gather data for the same day, for things happening 2 hours before the planned departure time.

First we need to parse the time in a good way to manipulate it efficiently

In [38]:
time_columns = ['CRS_DEP_TIME', 'DEP_TIME', 'CRS_ARR_TIME', 'ARR_TIME']
for c in time_columns:
    delay_all[c+'_CLEAN'] = delay_all[c].apply(lambda x: '0000' if x=='2400' else x)
    delay_all[c+'_CLEAN'] = delay_all.apply(lambda x: np.nan if (type(x[c+'_CLEAN']) is not str) or (type(x.FL_DATE) is not str) else x.FL_DATE+'_'+x[c+'_CLEAN'], axis=1)
    delay_all[c+'_CLEAN'] = delay_all[c+'_CLEAN'].apply(lambda x: x if (type(x) is not str) else datetime.strptime(x, '%Y-%m-%d_%H%M'))

delay_all['CRS_ARR_TIME_CLEAN'] = delay_all.apply(lambda x: x.CRS_ARR_TIME_CLEAN if x.CRS_ARR_TIME_CLEAN > x.CRS_DEP_TIME_CLEAN else x.CRS_ARR_TIME_CLEAN+timedelta(days=1), axis=1)
delay_all['ARR_TIME_CLEAN'] = delay_all.apply(lambda x: x.ARR_TIME_CLEAN if x.ARR_TIME_CLEAN > x.DEP_TIME_CLEAN else x.CRS_ARR_TIME_CLEAN+timedelta(days=1), axis=1)

For each line we want to gather all events happening at the departure or destination airport 2h before the actual CRS departure time, aggregated over the last hour and half day. 

For that we create a new key for the flights that will be considered in the 1h and 12h statistics. The idea is that for accelarating computation, we will synchronize different time periods, compute features for these periods and allocate a given period for delay prediction of a given flight. For example if we want to predict a flight on 2015.07.08 at 16:00 we will use the statistics of 2015.07.08 at 14:00 (1h hour period 2h before the flight) and 2015.07.08 morning (hour between 0 and 12)

In [39]:
def half_day_before_departue(x):
    if (x.hour >= 0) and (x.hour<2):
        return x.replace(hour=0, minute=0, second=0)-timedelta(hours=24)
    elif (x.hour>=2) and (x.hour<12):
        return x.replace(hour=0, minute=0, second=0)-timedelta(hours=12)
    elif (x.hour >= 12) and (x.hour<14):
        return x.replace(hour=12, minute=0, second=0)-timedelta(hours=24)
    elif (x.hour>=14) and (x.hour<24):
        return x.replace(hour=0, minute=0, second=0)-timedelta(hours=12)

In [40]:
delay_all['HOUR_OF_ARR'] = delay_all.ARR_TIME_CLEAN.apply(lambda x: x.replace(minute=0, second=0))
delay_all['12H_OF_ARR'] = delay_all.ARR_TIME_CLEAN.apply(lambda x: x.replace(hour=0, minute=0, second=0) if x.hour < 12 else x.replace(hour=12, minute=0, second=0))
delay_all['HOUR_BEFORE_DEP'] = delay_all.CRS_DEP_TIME_CLEAN.apply(lambda x: x.replace(minute=0, second=0)-timedelta(hours=2))
delay_all['12H_BEFORE_DEP'] = delay_all.CRS_DEP_TIME_CLEAN.apply(lambda x: half_day_before_departue(x))

In [41]:
data['HOUR_BEFORE_DEP'] = delay_all.HOUR_BEFORE_DEP
data['12H_BEFORE_DEP'] = delay_all['12H_BEFORE_DEP']

Then we can create our features for those two time periods

### 1) Hour of the day

In [42]:
data['HOUR_OF_THE_DAY'] = delay_all.CRS_DEP_TIME_CLEAN.map(lambda x: x.hour)

### 2) Numbered of delayed flights on the same route and inverse route 
Here we count how much low,medium and high delay we have during 1h and 12h on both the same route and the inverse route

In [43]:
hour_carrier_route_delay = (delay_all.groupby(['UNIQUE_CARRIER','ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','HOUR_OF_ARR'])
            .ARR_DELAY
            .agg({'LOW_DELAY': lambda x: len([elem for elem in x if elem<=5]),
                  'MID_DELAY': lambda x: len([elem for elem in x if (elem>5) and (elem <=15)]),
                  'HIGH_DELAY': lambda x: len([elem for elem in x if elem>15])}))
data = pd.merge(data, hour_carrier_route_delay, how='left', left_on=['UNIQUE_CARRIER','ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','HOUR_BEFORE_DEP'], right_index=True, sort=False)

In [44]:
data['ORIGIN_AIRPORT_ID_R'] = data['DEST_AIRPORT_ID']
data['DEST_AIRPORT_ID_R'] = data['ORIGIN_AIRPORT_ID']
data = pd.merge(data, hour_carrier_route_delay, how='left', left_on=['UNIQUE_CARRIER','ORIGIN_AIRPORT_ID_R','DEST_AIRPORT_ID_R','HOUR_BEFORE_DEP'], right_index=True, sort=False)

In [45]:
hour12_carrier_route_delay = (delay_all.groupby(['UNIQUE_CARRIER','ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','12H_OF_ARR'])
            .ARR_DELAY
            .agg({'LOW_DELAY_12H': lambda x: len([elem for elem in x if elem<=5]),
                  'MID_DELAY_12H': lambda x: len([elem for elem in x if (elem>5) and (elem <=15)]),
                  'HIGH_DELAY_12H': lambda x: len([elem for elem in x if elem>15])}))
data = pd.merge(data, hour12_carrier_route_delay, how='left', left_on=['UNIQUE_CARRIER','ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','12H_BEFORE_DEP'], right_index=True, sort=False)

In [46]:
data = pd.merge(data, hour12_carrier_route_delay, how='left', left_on=['UNIQUE_CARRIER','ORIGIN_AIRPORT_ID_R','DEST_AIRPORT_ID_R','12H_BEFORE_DEP'], right_index=True, sort=False)

### 3) Numbered of delayed flights in the departure and destination airport for the carrier

In [47]:
hour_carrier_air_delay = (delay_all.groupby(['UNIQUE_CARRIER','ORIGIN_AIRPORT_ID','HOUR_OF_ARR'])
            .ARR_DELAY
            .agg({'LOW_DELAY_AIR_CAR': lambda x: len([elem for elem in x if elem<=5]),
                  'MID_DELAY_AIR_CAR': lambda x: len([elem for elem in x if (elem>5) and (elem <=15)]),
                  'HIGH_DELAY_AIR_CAR': lambda x: len([elem for elem in x if elem>15])}))
data = pd.merge(data, hour_carrier_air_delay, how='left', left_on=['UNIQUE_CARRIER','ORIGIN_AIRPORT_ID','HOUR_BEFORE_DEP'], right_index=True, sort=False)
data = pd.merge(data, hour_carrier_air_delay, how='left', left_on=['UNIQUE_CARRIER','ORIGIN_AIRPORT_ID_R','HOUR_BEFORE_DEP'], right_index=True, sort=False)

In [48]:
hour12_carrier_air_delay = (delay_all.groupby(['UNIQUE_CARRIER','ORIGIN_AIRPORT_ID','12H_OF_ARR'])
            .ARR_DELAY
            .agg({'LOW_DELAY_AIR_CAR_12H': lambda x: len([elem for elem in x if elem<=5]),
                  'MID_DELAY_AIR_CAR_12H': lambda x: len([elem for elem in x if (elem>5) and (elem <=15)]),
                  'HIGH_DELAY_AIR_CAR_12H': lambda x: len([elem for elem in x if elem>15])}))
data = pd.merge(data, hour12_carrier_air_delay, how='left', left_on=['UNIQUE_CARRIER','ORIGIN_AIRPORT_ID','12H_BEFORE_DEP'], right_index=True, sort=False)
data = pd.merge(data, hour12_carrier_air_delay, how='left', left_on=['UNIQUE_CARRIER','ORIGIN_AIRPORT_ID_R','12H_BEFORE_DEP'], right_index=True, sort=False)

### 4) Numbered of delayed flights in the departure and destination aiport

In [49]:
hour_air_delay = (delay_all.groupby(['ORIGIN_AIRPORT_ID','HOUR_OF_ARR'])
            .ARR_DELAY
            .agg({'LOW_DELAY_AIR': lambda x: len([elem for elem in x if elem<=5]),
                  'MID_DELAY_AIR': lambda x: len([elem for elem in x if (elem>5) and (elem <=15)]),
                  'HIGH_DELAY_AIR': lambda x: len([elem for elem in x if elem>15])}))
data = pd.merge(data, hour_air_delay, how='left', left_on=['ORIGIN_AIRPORT_ID','HOUR_BEFORE_DEP'], right_index=True, sort=False)
data = pd.merge(data, hour_air_delay, how='left', left_on=['ORIGIN_AIRPORT_ID_R','HOUR_BEFORE_DEP'], right_index=True, sort=False)

In [50]:
hour12_air_delay = (delay_all.groupby(['ORIGIN_AIRPORT_ID','12H_OF_ARR'])
            .ARR_DELAY
            .agg({'LOW_DELAY_AIR_12H': lambda x: len([elem for elem in x if elem<=5]),
                  'MID_DELAY_AIR_12H': lambda x: len([elem for elem in x if (elem>5) and (elem <=15)]),
                  'HIGH_DELAY_AIR_12H': lambda x: len([elem for elem in x if elem>15])}))
data = pd.merge(data, hour12_air_delay, how='left', left_on=['ORIGIN_AIRPORT_ID','12H_BEFORE_DEP'], right_index=True, sort=False)
data = pd.merge(data, hour12_air_delay, how='left', left_on=['ORIGIN_AIRPORT_ID_R','12H_BEFORE_DEP'], right_index=True, sort=False)

In [51]:
data.shape

(896216, 92)

# III) Ground Truth
We will solve a multi-class classification problem with 3 types of delay
- Low delay: delay $\le$ 5min
- Medium delay: 5min $<$ delay $\le$ 15min
- Large delay: delay $>$ 15min
Note that these thresholds are purely arbitrary and based on my personal level of what is low or high delay. This can be easily changed to different values.

In [52]:
data['y'] = delay_all.ARR_DELAY
data.y[data.y <= 5] = 0
data.y[data.y > 15] = 2
data.y[(data.y != 0) & (data.y != 2) & (~np.isnan(data.y))] = 1

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# IV) Machine learning sets
Here we simply get numpy matrices for the training,cv and test sets as well as ground truth

First we drop information we used for joining tables but we don't want to use for learning

In [53]:
data.drop(['UNIQUE_CARRIER', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID','HOUR_BEFORE_DEP','12H_BEFORE_DEP','ORIGIN_AIRPORT_ID_R','DEST_AIRPORT_ID_R'], inplace=True, axis=1)

Then we split according to the previously explained time split, fill up the nan values in the features with -1 and remove all samples without groundtruth 

In [54]:
y = data.y.values
X = data.drop('y',axis=1).fillna(-1).values

In [55]:
X_train = X[:n_train_samples,:]
X_cv = X[n_train_samples:n_train_samples+n_cv_samples,:]
X_test = X[n_train_samples+n_cv_samples:,:]

y_train = y[:n_train_samples]
y_cv = y[n_train_samples:n_train_samples+n_cv_samples]
y_test = y[n_train_samples+n_cv_samples:]

In [56]:
X_train = X_train[~np.isnan(y_train)]
y_train = y_train[~np.isnan(y_train)].astype(int)
X_cv = X_cv[~np.isnan(y_cv)]
y_cv = y_cv[~np.isnan(y_cv)].astype(int)
X_test = X_test[~np.isnan(y_test)]
y_test = y_test[~np.isnan(y_test)].astype(int)

In [57]:
print 'Final shapes: train=',X_train.shape,', CV=',X_cv.shape,', test=',X_test.shape

Final shapes: train= (456453, 85) , CV= (201103, 85) , test= (204312, 85)


In [231]:
data.columns

Index([u'DEP_FLIGHTS', u'ARR_FLIGHTS', u'DEP_ROUTES', u'ARR_ROUTES',
       u'DEP_PASSENGERS', u'ARR_PASSENGERS', u'DEP_OCCUPANCY',
       u'ARR_OCCUPANCY', u'ROUTE_FREQ', u'ROUTE_IMP_DEP', u'ROUTE_IMP_ARR',
       u'ROUTE_PASSENGERS', u'ROUTE_IMP_PASS_DEP', u'ROUTE_IMP_PASS_ARR',
       u'ROUTE_OCCUPANCY', u'DEP_CARRIER_FLIGHTS', u'ARR_CARRIER_FLIGHTS',
       u'DEP_CARRIER_ROUTES', u'ARR_CARRIER_ROUTES', u'DEP_CARRIER_PASSENGERS',
       u'ARR_CARRIER_PASSENGERS', u'CARRIER_IMP_DEP', u'CARRIER_IMP_ROUTE_DEP',
       u'CARRIER_IMP_PASS_DEP', u'CARRIER_IMP_ARR', u'CARRIER_IMP_ROUTE_ARR',
       u'CARRIER_IMP_PASS_ARR', u'DEP_CARRIER_OCCUPANCY',
       u'ARR_CARRIER_OCCUPANCY', u'ROUTE_CARRIER_FREQ',
       u'ROUTE_CARRIER_PASSENGERS', u'ROUTE_CARRIER_OCCUPANCY',
       u'ROUTE_CARRIER_IMP_DEP', u'ROUTE_CARRIER_IMP_ARR',
       u'ROUTE_CARRIER_IMP_PASS_DEP', u'ROUTE_CARRIER_IMP_PASS_ARR',
       u'ROUTE_CARRIER_ROUTE_IMP', u'ROUTE_CARRIER_ROUTE_IMP_PASS',
       u'ROUTE_CARRIER_CARRIER_

# V) Feature selection using ANOVA
Use analysis of variance to see which feature best separate the ground truth 

In [215]:
F,P = f_classif(np.delete(X_train, 47, 1),y_train)

In [216]:
data.columns.drop('MONTH')[np.argsort(F)[-20:][::-1]]

Index([u'HIGH_DELAY_AIR_y', u'HIGH_DELAY_AIR_CAR_y', u'MID_DELAY_AIR_y',
       u'HIGH_DELAY_AIR_12H_y', u'HIGH_DELAY_AIR_CAR_12H_y',
       u'MID_DELAY_AIR_12H_y', u'MID_DELAY_AIR_CAR_y', u'HOUR_OF_THE_DAY',
       u'HIGH_DELAY_12H_y', u'HIGH_DELAY_AIR_x', u'HIGH_DELAY_12H_x',
       u'ARR_FLIGHTS', u'ARR_ROUTES', u'ARR_PASSENGERS',
       u'MID_DELAY_AIR_CAR_12H_y', u'ARR_OCCUPANCY', u'LOW_DELAY_AIR_12H_x',
       u'HIGH_DELAY_AIR_CAR_x', u'ROUTE_IMP_ARR',
       u'ROUTE_CARRIER_CARRIER_IMP_ARR'],
      dtype='object')

In [220]:
F,P = f_classif(X_train[:,:47],y_train)

In [223]:
data.columns[np.argsort(F)[-5:][::-1]]

Index([u'ARR_FLIGHTS', u'ARR_ROUTES', u'ARR_PASSENGERS', u'ARR_OCCUPANCY',
       u'ROUTE_IMP_ARR'],
      dtype='object')

In [225]:
F[0]

487.76070261867602

# VI) Feature importance using Random Forest
In this section, we will look into the first classification accuracy results and which features dominates the classification process in Random Forest to derive which features are more influential on the delay

In [107]:
model_rf = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=5, 
                               min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                               max_features='auto', max_leaf_nodes=None, min_impurity_split=1e-07,
                               bootstrap=True, oob_score=False, n_jobs=-1, random_state=2016, 
                               verbose=1, warm_start=False, class_weight=None)

In [108]:
model_rf.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   11.5s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False,
            random_state=2016, verbose=1, warm_start=False)

In [109]:
y_pred = model_rf.predict_proba(X_test)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.3s finished


In [110]:
print 'Log loss error=', log_loss(y_test, y_pred)

Log loss error= 0.92356565029


In [113]:
y_same = np.ones((y_pred.shape[0],3))*(1/3.0)

In [114]:
print 'Random guess log loss error=', log_loss(y_test, y_same)

Log loss error= 1.09861228867


In [116]:
idmax = np.argsort(model_rf.feature_importances_)[-10:][::-1]
idmax

array([84, 78, 82, 71, 83, 65, 77, 76,  1,  5, 45,  3,  7, 24, 48, 66, 46,
       72, 64, 75, 16, 44, 58, 18, 20, 26, 74, 28])

In [117]:
model_rf.feature_importances_[idmax]

array([ 0.11317839,  0.08728532,  0.07348048,  0.0635536 ,  0.05925539,
        0.05619052,  0.05057001,  0.04956328,  0.04244013,  0.04237969,
        0.04051474,  0.03750147,  0.03099607,  0.02054762,  0.0202518 ,
        0.01818879,  0.01723635,  0.01423387,  0.01377152,  0.01358571,
        0.01334697,  0.01176555,  0.01159248,  0.01088431,  0.00954504,
        0.00682398,  0.00582341,  0.00581226])

In [118]:
data.columns[idmax]

Index([u'HIGH_DELAY_AIR_12H_y', u'HIGH_DELAY_AIR_y', u'MID_DELAY_AIR_12H_y',
       u'HIGH_DELAY_AIR_CAR_12H_y', u'LOW_DELAY_AIR_12H_y',
       u'HIGH_DELAY_AIR_CAR_y', u'MID_DELAY_AIR_y', u'LOW_DELAY_AIR_y',
       u'ARR_FLIGHTS', u'ARR_PASSENGERS', u'long_y', u'ARR_ROUTES',
       u'ARR_OCCUPANCY', u'CARRIER_IMP_ARR', u'HOUR_OF_THE_DAY',
       u'MID_DELAY_AIR_CAR_y', u'DAY', u'LOW_DELAY_AIR_CAR_12H_y',
       u'LOW_DELAY_AIR_CAR_y', u'HIGH_DELAY_AIR_x', u'ARR_CARRIER_FLIGHTS',
       u'lat_y', u'HIGH_DELAY_12H_y', u'ARR_CARRIER_ROUTES',
       u'ARR_CARRIER_PASSENGERS', u'CARRIER_IMP_PASS_ARR', u'MID_DELAY_AIR_x',
       u'ARR_CARRIER_OCCUPANCY'],
      dtype='object')

Here we run random forest only on general feature

In [229]:
model_rf.fit(X_train[:,:47], y_train)
y_pred = model_rf.predict_proba(X_test[:,:47])
print 'Log loss error=', log_loss(y_test, y_pred)
data.columns[np.argsort(model_rf.feature_importances_)[-5:][::-1]]

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    6.8s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.3s finished


Log loss error= 0.945074919098


Index([u'ARR_FLIGHTS', u'ARR_PASSENGERS', u'long_y', u'ARR_ROUTES',
       u'ARR_OCCUPANCY'],
      dtype='object')

And then we do the opposite, we run only on daily features

In [232]:
model_rf.fit(X_train[:,49:], y_train)
y_pred = model_rf.predict_proba(X_test[:,49:])
print 'Log loss error=', log_loss(y_test, y_pred)
data.columns[49+np.argsort(model_rf.feature_importances_)[-5:][::-1]]

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    6.6s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.3s finished


Log loss error= 0.918437564556


Index([u'HIGH_DELAY_AIR_12H_y', u'HIGH_DELAY_AIR_y', u'MID_DELAY_AIR_12H_y',
       u'HIGH_DELAY_AIR_CAR_12H_y', u'LOW_DELAY_AIR_12H_y'],
      dtype='object')

# VII) Recursive feature elimination with cross validation
Here we will try to recursively eliminate features to keep only those which provide the best cv score. For that we take a simple fast linear classifier as backbone and run REFCV.

In [208]:
model_sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, l1_ratio=0.15, 
                      fit_intercept=True, n_iter=10, shuffle=True, verbose=0, 
                      epsilon=0.1, n_jobs=1, random_state=None, learning_rate='optimal', 
                      eta0=0.0, power_t=0.5, class_weight=None, warm_start=False, average=False)

In [145]:
selector_simple = RFE(model_sgd, n_features_to_select=10, step=1, verbose=0)
selector_simple.fit(np.vstack([X_train, X_cv]), np.hstack([y_train,y_cv]))

RFE(estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=10, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False),
  n_features_to_select=10, step=1, verbose=0)

In [188]:
data.columns[np.hstack([selector_simple.support_,np.array([False])])]

Index([u'HIGH_DELAY_12H_x', u'LOW_DELAY_12H_x', u'MID_DELAY_AIR_CAR_x',
       u'LOW_DELAY_AIR_CAR_y', u'HIGH_DELAY_AIR_CAR_y', u'MID_DELAY_AIR_CAR_y',
       u'HIGH_DELAY_AIR_x', u'MID_DELAY_AIR_y', u'HIGH_DELAY_AIR_y',
       u'MID_DELAY_AIR_12H_y'],
      dtype='object')

Here we run feature elimination only on general features

In [ ]:
model_sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, l1_ratio=0.15, 
                      fit_intercept=True, n_iter=10, shuffle=True, verbose=0, 
                      epsilon=0.1, n_jobs=1, random_state=None, learning_rate='optimal', 
                      eta0=0.0, power_t=0.5, class_weight=None, warm_start=False, average=False)
selector_simple_general = RFE(model_sgd, n_features_to_select=10, step=1, verbose=0)
selector_simple_general.fit(np.vstack([X_train[:,:47], X_cv[:,:47]]), np.hstack([y_train,y_cv]))

In [228]:
data.columns[np.hstack([selector_simple_general.support_,np.array([False]*(X_train.shape[1]-46))])]

Index([u'ARR_ROUTES', u'ROUTE_FREQ', u'ARR_CARRIER_FLIGHTS',
       u'DEP_CARRIER_ROUTES', u'ARR_CARRIER_ROUTES', u'ARR_CARRIER_PASSENGERS',
       u'ROUTE_CARRIER_FREQ', u'lat_x', u'long_x', u'long_y'],
      dtype='object')

Here we run REFCV

In [ ]:
model_sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, l1_ratio=0.15, 
                      fit_intercept=True, n_iter=10, shuffle=True, verbose=0, 
                      epsilon=0.1, n_jobs=1, random_state=None, learning_rate='optimal', 
                      eta0=0.0, power_t=0.5, class_weight=None, warm_start=False, average=False)

cv_fold = np.ones(X_train.shape[0]+X_cv.shape[0])
cv_fold[:X_train.shape[0]]=-1
cv = PredefinedSplit(cv_fold)

selector = RFECV(model_sgd, step=1, cv=cv, verbose=0, n_jobs=1)
selector.fit(np.vstack([X_train, X_cv]), np.hstack([y_train,y_cv]))

In [ ]:
selector.n_features_

In [ ]:
data.columns[np.hstack([selector.support_,np.array([False])])]